In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS
%matplotlib inline

In [2]:
list_files = os.listdir('datasets/history-scottish-witchcraft/')

In [3]:
# Put all data files into a dictionary of dataframes for easy access
def create_dfs(list_files):
    dfs_dict = {}
    for filename in list_files:
        if filename[-4:] == '.csv':
            df_name = filename[4:-4]
            dfs_dict[df_name] = pd.read_csv('datasets/history-scottish-witchcraft/{}'.format(filename))
    return dfs_dict

In [4]:
all_dfs = create_dfs(list_files)

In [5]:
all_dfs.keys()

['MovestoHLA',
 'CalendarCustom',
 'Imprisonment',
 'MentionedAsWitch',
 'RitualObject',
 'Commission',
 'ReligiousMotif',
 'DevilAppearance',
 'Appeal',
 'OtherNamedwitch',
 'ShapeChanging',
 'Complaint',
 'WitchesMeetingPlace',
 'Torture',
 'Elf_FairyElements',
 'WeatherModification',
 'Ref_Parish',
 'Trial_Person',
 'Case',
 'PropertyDamage',
 'PrevCommission',
 'DemonicPact',
 'Ordeal',
 'Person',
 'Denunciation',
 'Trial',
 'WhiteMagic',
 'MusicalInstrument',
 'Reference',
 'Source',
 'Confession',
 'CounterStrategy',
 'Malice',
 'Case_person',
 'Accused',
 'LinkedTrial',
 'Accused_family',
 'OtherCharges']

In [6]:
accused = all_dfs['Accused']

In [19]:
malice = all_dfs['Malice']

In [21]:
malice.Causeofmalice.unique()

array(['Failed business interaction', 'Debt', 'Jealousy of Neighbours',
       'Revenge', 'Grudge', 'Slander', 'Social slight', 'Physical insult',
       'Exclusion from social events', 'Refusal of alms', nan,
       'Business interactions', 'Failed healing',
       'Revenge for being called a witch', 'Verbal Insult', 'Taunting',
       'Envy'], dtype=object)

In [22]:
confession = all_dfs['Confession']

In [26]:
torture = all_dfs['Torture']

In [27]:
torture.Torturetype.unique()

array(['Burning feet', 'Various', 'Sleep Deprivation', 'Bound with ropes',
       'Whip', 'Hanging by thumbs', nan, 'Stocks', 'Irons', 'Bow strings',
       'Wedges on the shins', 'Cashielaws', 'Haircloth', 'Tied to pole'], dtype=object)

In [28]:
imprisonment = all_dfs["Imprisonment"]

In [30]:
devil_appearance = all_dfs['DevilAppearance']

In [31]:
devil_by_type = devil_appearance.groupby("Devil_Type")['CaseRef'].count()

In [32]:
devil_by_type.sort_values(ascending=False)

Devil_Type
Male                      250
Animal Devil               60
Female                     26
Spirit                     17
Unspecified Devil           9
Male Fairy                  9
Female Fairy                6
Ghost                       5
Fairy                       4
Inanimate Object Devil      2
Human Devil                 2
Other Demon                 1
Insect Devil                1
Child Devil                 1
Baby                        1
Name: CaseRef, dtype: int64

In [34]:
case = all_dfs['Case']

In [51]:
case["OtherChargesNotes"].unique()

array([nan,
       'The details in the presbytery records are all about charming. Witchcraft is not mentioned.',
       'Incest with his mother and sister for two years',
       'charged with charming in RPC p. 104',
       'In trouble in 1617 charged with adultery.',
       "Also accused of having sex (and a baby) with her sister's husband.",
       'She confessed to both',
       'Presbytery accepted that he was guilty of charming and witchcraft in 1643 and ordered him to make public satisfaction.',
       'Named as witch and charmer by Janet Coutts',
       'Presbytery minutes note that she had a written book of her pretended prophecies.',
       'no details',
       'Accused of murdering two women by poisoning and also of murdering her own (first?) husband.',
       'Entry in presbytery notes that accused had used charming many times previously.',
       'Main accusation according to presbytery was blasphemy and devilish delusions which later became devilry, blasphemy and witchcraf

In [53]:
case['OtherMaleficiaNotes'].unique()

array([nan,
       'Said to have been part of the plot to sink the ships of James VI and his wife Anne of Denmark. Claimed by others that they raised storms and winds.',
       'Claimed by other confessing witches that she burned down the Mill; destroyed ships during a herring drove',
       'Claimed by another confessing witch that she burned down a mill; destroyed ships during a herring drove',
       "Part of the witches' crew that tried to kill Ann of Denmark.",
       'Allegedly sunk a ship.',
       'It was claimed by others that she was involved in trying to sink the royal ships.',
       "Supposedly raised a storm in Newhaven harbour.  She predicted that there wouldn't be enough salt for the Dunbar herring drove.",
       "She was accused of being an accomplice to Gourley's (c/egd/308) firestarting.  She claimed that she and Gourley took 5 burning coals and put them in places of the house that burnt down.",
       'burned a mill; destroyed ships during a herring drove',
       

In [54]:
case['CaseNotes']

0                          certain persons from Haymouth.
1                                                     NaN
2                                                     NaN
3                                                     NaN
4                                                     NaN
5                                                     NaN
6       A very complicated case that moved back and fo...
7                                                     NaN
8                                                     NaN
9                                                     NaN
10                                                    NaN
11                                                    NaN
12                                                    NaN
13                                                    NaN
14                                                    NaN
15                                                    NaN
16                                                    NaN
17            

In [60]:
case['DefenseNotes'].unique()

array([nan,
       'Appealed to Privy Council to change the location of their trial to Edinburgh, rather than Haddington. Questioned the competency of the local commissioners. Appeal denied.',
       "A doctor testified that she had symptoms of 'hyprocondriack distraction'",
       'No words or deeds; admits to blasphemy',
       'She used the common defence strategy of trying to discredit the witnesses.',
       'Claimed that the only thing in her dittay that was an offence of any kind was the ritual with the nettle and the urine which the defence argued was a superstitious rite not witchcraft.  Lots of detailed legal pleadings, very rich on this.',
       'Argued that no words or deeds were alleged; use of ridicule to challenge witnesses',
       'Accused petitioned the PC on the grounds that she was too old to be in prison and that there were no grounds to the accusation other than the word of an executed person. She complained that she had had a good name and had no bad reputation.

In [68]:
case['Chronicle'].unique()

array([ nan])

In [72]:
case['Complaint'].unique()

array([nan, 'against a witch-pricker',
       'bailies complaint about friends of witches visiting them in ward',
       'About a common ritual'], dtype=object)

In [76]:
case['Other'].unique()

array([nan, 'Kirk Session minutes', 'Burgh Council Minutes',
       'Presbytery entry', 'kirk session', 'Presbytery records',
       'Presbytery minutes', 'kirk session records',
       'specification of witchcraft under jurisdiction of circuit courts',
       'Arrest of C. Cauldwell (alias John Dickson) as a witchpricker',
       'Court Book of Orkney', 'Court book of Shetland',
       'Statute forbidding the concealment of witchcraft',
       'Shetland law forbidding the concealment of witchcraft',
       'John Kincaid, witch pricker examined', 'General Assembly policy',
       'Ayre Burgh Accounts', 'Some persons in Dreghorn',
       "Minister's announcement to pursue witches",
       'Church of Scotland petition to Parliament',
       'Presbytery Querry re: witch finders',
       'for tryal and torture of witches',
       'Bailie requests ministers to wait upon 12 condemed witches',
       'Presbytery request for commission',
       'Bailie complaint to Presbytery about their impro

In [79]:
case['Charnotes'].unique()

array([nan,
       'Part of the North Berwick witch hunt.  It involved many people and many accustions about witches meetings, specifically two separate attempts to sink the royal ships.  The first to sink Queen Anne on her way home, second to trouble King James after he went to get her in Denmark.',
       'Part of the North Berwick witch hunt.  It involved many people and many accustions about witches meetings, specifically two separate attempts to sink the royal ships.  The first to sink Queen Anne on her way home, the second to trouble King James after he went to get her in Denmark.',
       'The main accusations appear to relate to healing rituals and advice that did not work and the patients died rather than death as a result of maleficium. No demonic features.',
       'Likely to be demonic but not enough information.',
       'Demonic pact and charming confessed so included but few details.',
       'Pact and charming confessed but few details.',
       'Demonic pact and charmi

In [81]:
case['DevilNotes'].unique()

array([nan, "Said by others to have met the devil as a 'mekle black man'",
       'Conversing with the Devil',
       "Others confessed that she met the devil as a 'mekle black man'.",
       'Baptised a cat.', 'No details of pact.', 'No pact details',
       'no details about pact recorded.',
       'paction confessed but no details.',
       'Pact confessed but no details.',
       'Presbytery notes confessed paction with the devil.',
       'Few details but presbytery notes she confessed paction with the devil.',
       'Her servant saw a vision of a man in a white shirt with a tail.',
       'Confessed that he had seen the fairies in different forms but did not describe them.',
       'Dalkeith presbytery note the accused confessed to paction, witchcraft and charming.',
       'Devil gave her the power to heal.',
       "Denounced Bothwell.  She said the devil would kill the king for Lord Bothwell's sake.",
       "Had a 'familiar spreit'.  Hearsay evidence given in by a woman who 

In [83]:
case['MeetingNotes'].unique()

array([nan,
       "See Agnes Sampson's case for description of North Berwick meeting (c/egd/63).  See also Napier (c/egd/103) for more details about the North Berwick meeting. Meeting said to have taken place at the kirk at North Berwick, and a trump was played as a musical instrument.",
       "Claimed by others that she was part of the planning of the conspiracy to sink Queen Anne'sI ships on the way from Denmark.  Said to have been at meetings at Beigis Tod's house and the salt pans. See Agnes Sampson's case for description of North Berwick meeting (c/egd/63).  She was supposedly part of the baptism of a cat.",
       "See Agnes Sampson's case for description of North Berwick meeting (c/egd/63).  See also Napier (c/egd/103) for more details about the North Berwick meeting. Claimed that she attended meeting at the kirk in North Berwick and that a trump was played.",
       "Prior to the North Berwick description there is a detailed account of who was supposed to do what in preparati

In [85]:
case['FolkNotes'].unique()

array([nan,
       "In other accused's confession it was said that they had used corpse and corpse powder, and had met at halloween.",
       'She was included in the ritual to tie joints to the feet of a cat and then cast it into the sea to raise storms.',
       'Claimed by others that she was involved in using a corpse and powder from a corpse and attended meetings at Halloween.',
       'This case has some interesting details about the herbs and other ingredients that she used in salves. These included rippell grasse (ripple grass or robwort plantain), waffron leaf(?), plantain feather(?), finglie (?finkle or fennel), and hyssop(?). She denied she had used foxtree leaves (foxglove). She also claimed that she had her skill from the possession of a book, that had belonged to her father and grandfather. She said it was a 1000 years old but as she could not read it her son, Adam Bell, would read out bits to her. The book was taken from her during the ministry of William Couper who was 

In [91]:
trial = all_dfs['Trial']

In [95]:
trial['TrialNotes'].unique()

array([nan, 'No trial details.', 'No details', 'no trial details.',
       'No details.',
       'Did not appear for trial as she had escaped. Sentence of excommunication passed.',
       'No details of trial.',
       'There is a note in the presbyteryon 24/7/1550  that witches were to be tried on the following Monday in Renfrew but no details of outcome.',
       '24/7/1650, note in presbytery records that the trial was to be held the following Monday in Renfrew but no details of outcome.',
       'Note in presbytery records on 24/7/1650 that a trial was to be held the following Monday in Renfrew, but no record of verdict.',
       'Note in presbytery records on 24/7/1650 that a trial was to be held on the following Monday in Renfrew but no details about outcome.',
       'Note in presbytery records on 17/3/1697 that ministers and commissioners to meet tomorrow in Renfrew for trial (?).  Trial started on 13/4/1697 but was continued until 12/5/1697 it lasted until the 19th.',
       '

In [183]:
def generate_cloud(text, img_path, out_file_path, background_color = "white", size = 15):
    with open(img)
    img_mask = np.array(Image.open(img_path))
    stopwords = set(STOPWORDS)
    wc = WordCloud(background_color=background_color, max_words=3000, mask=img_mask,
                   stopwords=stopwords)
    wc.generate(text)
    # store to file
    wc.to_file(out_file_path)
    # show
    plt.figure(figsize=(size,size))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    plt.show()

In [178]:
accused_text = pd.DataFrame(accused['Notes'])

accused_text = accused_text.dropna()

accused_text = ' '.join([w.lower() for w in accused_text.Notes.values.flatten().tolist()])

In [ ]:
generate_cloud(accused_text, "images/witch.png", "images/witch_cloud.png")

In [115]:
devil_notes = pd.DataFrame(case['DevilNotes'])
devil_notes = devil_notes.dropna()
devil_text = " ".join([w.lower() for w in devil_notes.values.flatten().tolist()])
devil_text

In [ ]:
generate_cloud(devil_text, "images/DevilSilhouette.png", "images/devil_cloud.png")

In [143]:
# images taken from http://www.fun-lover.com/graphic-shop/Masks/index.php?page=&sort=&perPage=&album=Halloween
# and https://openclipart.org/detail/233745/devil-silhouette